<a href="https://colab.research.google.com/github/Arup13455/Python-Projects/blob/main/LMS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install IPython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.5 MB/s eta 0:00:00


In [10]:
import pandas as pd
import datetime as dt
from IPython.display import display, HTML
import time

# Load the Excel file
def load_data(file_name="/content/lib_data2.xlsx"):
    try:
        df = pd.read_excel(file_name, sheet_name='Sheet1', engine='openpyxl')
    except FileNotFoundError:
        print(f"File {file_name} not found.")
        df = pd.DataFrame(columns=['ID', 'Title', 'Author', 'Genre', 'Available', 'BorrowDate', 'ReturnDate', 'LateFine'])
    return df

# Save the DataFrame to the Excel file
def save_data(df, file_name="/content/lib_data2.xlsx"):
    with pd.ExcelWriter(file_name, engine='openpyxl') as writer:
        df.to_excel(writer, sheet_name='Sheet1', index=False)

# View all books
def view_books():
    df = load_data()
    print(df)

# Add a new book
def add_book(id, title, author, genre):
    df = load_data()
    new_book = pd.DataFrame([[id, title, author, genre, 'Yes', None, None, 0]], columns=df.columns)
    df = pd.concat([df, new_book], ignore_index=True)
    save_data(df)
    print("Book added successfully.")

# Borrow a book by genre
def borrow_book_by_genre():
    genre = input("Enter the genre of the book you want to borrow: ")
    df = load_data()
    available_books = df[(df['Genre'].str.contains(genre, case=False, na=False)) & (df['Available'] == 'Yes')]

    if available_books.empty:
        print(f"No available books found in the genre: {genre}")
    else:
        print("Available books:")
        print(available_books)

        try:
            book_id = int(input("Enter the ID of the book you want to borrow: "))
            if book_id in available_books['ID'].values:
                df.loc[df['ID'] == book_id, 'Available'] = 'No'
                df.loc[df['ID'] == book_id, 'BorrowDate'] = dt.datetime.now()
                save_data(df)
                print("Book borrowed successfully.")
            else:
                print("Invalid book ID.")
        except ValueError:
            print("Invalid input. Please enter a valid book ID.")

# Return a book
def return_book(id):
    df = load_data()
    if id in df['ID'].values:
        df.loc[df['ID'] == id, 'Available'] = 'Yes'
        df.loc[df['ID'] == id, 'ReturnDate'] = dt.datetime.now()

        borrow_date = df.loc[df['ID'] == id, 'BorrowDate'].values[0]
        if pd.notnull(borrow_date):
            borrow_date = pd.to_datetime(borrow_date)
            days_borrowed = (dt.datetime.now() - borrow_date).days
            if days_borrowed > 3:
                late_fine = (days_borrowed - 3) * 2
                df.loc[df['ID'] == id, 'LateFine'] = late_fine
                print(f"Late fine for book ID {id}: ${late_fine}")

        save_data(df)
        print("Book returned successfully.")
    else:
        print("Book ID not found.")

# Send notifications for overdue book

def send_notifications():
    df = load_data()
    overdue_books = df[(df['Available'] == 'No') & (pd.to_datetime(df['BorrowDate']) < (dt.datetime.now() - dt.timedelta(days=1)))]

    if not overdue_books.empty:
        for index, row in overdue_books.iterrows():
            borrow_date = pd.to_datetime(row['BorrowDate'])
            days_overdue = (dt.datetime.now() - borrow_date).days - 1
            message = f"Book ID {row['ID']} is overdue by {days_overdue} days. Late fine: ${days_overdue * 2}"
            display(HTML(f'<div style="background-color:yellow; padding:10px;">{message}</div>'))
            time.sleep(1)  # Delay between notifications
# Main menu with automatic notification
def main():
    while True:
        print("\nLibrary Management System")
        print("1. View Books")
        print("2. Add Book")
        print("3. Borrow Book")
        print("4. Return Book")
        print("5. Display Notifications")
        print("6. Exit")

        send_notifications()  # Automatically send notifications each time the menu is displayed

        choice = int(input("Enter your choice: "))

        if choice == 1:
            view_books()
        elif choice == 2:
            id = int(input("Enter book ID: "))
            title = input("Enter book title: ")
            author = input("Enter book author: ")
            genre = input("Enter book genre: ")
            add_book(id, title, author, genre)
        elif choice == 3:
            borrow_book_by_genre()
        elif choice == 4:
            id = int(input("Enter book ID to return: "))
            return_book(id)
        elif choice == 5:
            send_notifications()
        elif choice == 6:
            break
        else:
            print("Invalid choice. Please try again.")

if __name__ == "__main__":
    main()











Library Management System
1. View Books
2. Add Book
3. Borrow Book
4. Return Book
5. Display Notifications
6. Exit
Enter your choice: 1
   ID              Title            Author             Genre Available  \
0   1   The Great Gatsby           F.Scott           Fiction       Yes   
1   2               1984     George Orwell          Dystopia       Yes   
2   3         Moboy Dick  Herm an Melville           Fiction       Yes   
3   4        Core Python  Dr. Nageswar Rao            Coding       Yes   
4   5             Core C     Danish Ritche            Coding       Yes   
5   6          Core Java        Dr. Ganesh            Coding       Yes   
6   7  Rich Dad poor Dad           W.Smith  Money Management       Yes   
7  23     Cyber Security        Dr. sewgsr         CyberTech       Yes   
8   8                 OS           Dr.erew  Operating system       Yes   
9   9          GopalBhar         Nabogopal             Comix       Yes   

               BorrowDate              ReturnDat